In [2]:
import cv2
import numpy as np
DIGITS_LOOKUP = {
	(1, 1, 1, 0, 1, 1, 1): 0,
	(1, 1, 0, 0, 0, 0, 0): 1,
	(1, 0, 1, 1, 1, 0, 1 ): 2,
	(1, 0, 1, 1, 0, 1, 1): 3,
	(0, 1, 1, 1, 0, 1, 0): 4,
	(1, 1, 0, 1, 0, 1, 1): 5,
	(1, 1, 0, 1, 1, 1, 1): 6,
	(1, 0, 1, 0, 0, 1, 0): 7,
	(1, 1, 1, 1, 1, 1, 1): 8,
	(1, 1, 1, 1, 0, 1, 0): 9
}



In [3]:
def sort_contours(cnts, method="left-to-right"):
	# initialize the reverse flag and sort index
	reverse = False
	i = 0
	# handle if we need to sort in reverse
	if method == "right-to-left" or method == "bottom-to-top":
		reverse = True
	# handle if we are sorting against the y-coordinate rather than
	# the x-coordinate of the bounding box
	if method == "top-to-bottom" or method == "bottom-to-top":
		i = 1
	# construct the list of bounding boxes and sort them from top to
	# bottom
	boundingBoxes = [cv2.boundingRect(c) for c in cnts]
	(cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
		key=lambda b:b[1][i], reverse=reverse))
	# return the list of sorted contours and bounding boxes
	return (cnts, boundingBoxes)
def imagsdo(img):
    imggray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(imggray, (5, 5), 0)
    _,thresh = cv2.threshold(imggray,110, 255,cv2.THRESH_BINARY)
    kernel = np.ones((7,7), np.uint8) 
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)
    img2=img_dilation.copy()
    contours,_ =cv2.findContours(img2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    img3=cv2.drawContours(img2,contours,-1,(0,255,255),15)
    return img2
    

In [4]:
def imago(path):
    img=cv2.imread(path)
    imggray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(imggray, (5, 5), 0)
    _,thresh = cv2.threshold(imggray,110, 255,cv2.THRESH_BINARY)
    kernel = np.ones((7,7), np.uint8) 
    img_dilation = cv2.dilate(thresh, kernel, iterations=1) 
    img2=img_dilation.copy()
    contours,_ =cv2.findContours(img2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    contours= sort_contours(contours)[0]
    digits=[]
    for c in contours:
        (x,y,w,h)=cv2.boundingRect(c)
        roi=img_dilation[y:y+h,x:x+w]
        (roiH, roiW) = roi.shape
        (dW, dH) = (int(w * 0.333), int(h * 0.26))
       # print(roi.shape,dW,dH)
        dHC = int(roiH * 0.05)
        if w<30:
            segments=[
                ((0,0),(w,h//2)),
                ((0,h//2),(w,h))
            ]
        else:    
            segments = [
            ((0, 0), (w, dH)),	# top
            ((0, 0), (dW, h // 2)),	# top-left
            ((w - dW, 0), (w, h // 2)),	# top-right
            ((0, (h // 2) - dHC) , (w, (h // 2) + dHC)),
            ((0, h // 2), (dW, h)),	# bottom-left
            ((w - dW, h // 2), (w, h)),	# bottom-right
            ((0, h - dH), (w, h))
            ]
        #print(len(segments))
        on = [0] * 7
        #print(len(on))
        for (i, ((xA, yA), (xB, yB))) in enumerate(segments):
            segROI = roi[yA:yB, xA:xB]
            total = cv2.countNonZero(segROI)
            area = (xB - xA) * (yB - yA)
            #print(total / float(area))
            if total / float(area) > 0.56:
                on[i]= 1
                
                #print(1)
            #else:
               #print(0)        
            #cv2.imshow('imf',segROI)
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()
        digit = DIGITS_LOOKUP[tuple(on)]
        digits.append(digit)
   # print(digits)
    x = int("".join(map(str, digits)))
    return x


In [5]:
images=['Num1.jpg','Num2.jpg','A.jpg','B.jpg','C.jpg','D.jpg']
dig=list(map(imago,images))
dig

[26, 15, 41, 390, 11, 21]

In [6]:
def showimage(path):
    img=cv2.imread(path)
    cv2.imshow("show",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [19]:
for i in images:
    showimage(i)

In [7]:
my_dic=dict(zip(images,dig))
my_dic

{'Num1.jpg': 26,
 'Num2.jpg': 15,
 'A.jpg': 41,
 'B.jpg': 390,
 'C.jpg': 11,
 'D.jpg': 21}

In [8]:
my_dic['Num1.jpg']+my_dic['Num2.jpg']
#img=cv2.imread(path)

41

In [9]:
showimage(images[dig.index(my_dic['Num1.jpg']^my_dic['Num2.jpg'])])
#xor operation

In [10]:
showimage(images[dig.index(my_dic['Num1.jpg']+my_dic['Num2.jpg'])])
#additon

In [11]:
showimage(images[dig.index(my_dic['Num1.jpg']*my_dic['Num2.jpg'])])
#multiplication

In [12]:
showimage(images[dig.index(my_dic['Num1.jpg']-my_dic['Num2.jpg'])])
#subtraction

In [15]:
image_set2=['Num1_1.jpg','Num2_1.jpg','A_1.jpg','B_1.jpg','C_1.jpg','D_1.jpg']
dig2=list(map(imago,image_set2))
dig2

[84, 79, 6636, 5, 163, 27]

In [20]:
for i in image_set2:
    showimage(i)